---
title: "Estadísticos en regresión lineal por variable"
description: "Revisión paso a paso de cómo calcular el p-valor de una variable en una regresión lineal"
description-meta: "Guía paso a paso de cómo calcular el p-valor de una variable en una regresión lineal"
author: "Leonardo Hansa"
date: "2024-08-16"
categories: [datos]
execute: 
  echo: true
  message: false
  warning: false
freeze: true
---


## Intro y datos
Voy a revisar aquí cómo se calcula el p-valor de una variable en una regresión lineal, porque es una cosa que siempre se me olvida. Y quiero tenerlo a mano.

Además, lo voy a hacer en Python porque me gusta sufrir.


In [ ]:
#| label: libs
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import matplotlib.pyplot as plt

Me invento unos datos que me sirvan para ajustar una regresión lineal: 

$$
y = \beta_0 + \beta_1 \cdot x + \epsilon,
$$

donde $\beta_0 = - 5$, $\beta_1 = 3$ y $x ~ \mbox{Poisson(12)}$ y $\epislon ~ \cal{N}(0, 10)$.


In [ ]:
#| label: generate-data
nobs = 10000
x = np.random.poisson(12, nobs)
noise = np.random.normal(0, 10, nobs)

y = 3 * x - 5 + noise

Ahora ajusto la regresión lineal con **statsmodels**. ¿Se puede hacer con **sklearn**? Pues imagino que sí, pero necesitaré luego acceder a detalles del modelo a los que no sé acceder con sklearn.


In [ ]:
#| label: fit-model
X = sm.add_constant(x)
model = sm.OLS(y, X).fit()
print(model.summary())

Para calcular el p-valor de $x$ en el modelo necesito el estadístico $t$. Para eso necesito, aparte del valor del coeficiente, el error estándar de la variable. 

## Error estándar

Este error estándar relaciona la varianza de la variable con la varianza de los errores: 

- Si una variable tiene poca varianza y los errores tienen mucha, la variable está aportando poco al modelo. 
- Si una variable tiene una varianza alta, estará contribuyendo más al modelo. 
- Si un modelo tiene errores con varianza baja, el error estándar de la variable tenderá a ser bajo.

> Cuanto mayor sea el error estándar, menor siginificatividad tendrá la variable. Por lo tanto, con un varianza del error pequeña, más fácil será que la variable aparezca significativa.


In [ ]:
#| label: standard-error
standard_error = np.sqrt(np.sum(model.resid ** 2) / (len(x) - 2))
standard_error_x = standard_error / np.sqrt(nobs)  / np.std(x)

El error estándar de los residuos es `{python} standard_error` y el de la variable es `{python} standard_error_x`.

> El error estándar del intercept es más lío de calcular e interpretar; la idea intuitiva es que tiene en cuenta la varianza de todas las variables explicativas a la vez. 


In [ ]:
#| label: standard-error-int
standard_error * np.sqrt(1 / nobs + np.mean(x) ** 2 / np.sum((x - np.mean(x)) ** 2))

## Estadístico $t$
El estadístico $t$ se calcula como 

$$
t = \frac{\mbox{coef}\ x}{\mbox{std error}\ x}.
$$ 

- Si la variable tiene un error pequeño, el estadístico crece (eso es bueno para la significatividad de la variable). 
- A su vez, si el coeficiente es alto (en valor absoluto), entonces también crece. 
- El estadístico decrece con coeficientes bajos o con errores altos (es decir, la variable parecerá no significativa).


In [ ]:
# label: t-stat
t_stat_x = np.abs(model.params[1]) / standard_error_x
t_stat_x

## p-valor
Ahora hay que ver qué valor es esperable para el estadístico $t$. 

Pon que trabajas al nivel de confianza del 90%. Hace falta calcular el $t$ que deja el 5% a un lado de la distribución, y el que lo deja al otro lado. Como la distribución es simétrica, solo calculo uno:


In [ ]:
percentile = stats.t.ppf(0.95, nobs - 2)
percentile

Ahora tienes la distribución de $t$. Si $t$ cae en el área sombreada (el 10% extremo) entonces la probabilidad de observar tus datos en el supuesto de que la variable no sea significativa ($H_0$ o $\beta_1 = 0$) es menor de 0,10.


In [ ]:
#| label: t-distribution
xx = np.linspace(-4, 4, 1000)
yy = stats.t.pdf(xx, nobs - 2)
plt.plot(xx, yy, color='#800080')
# plot the t distribution and fill the 5% outside 
plt.fill_between(xx, yy, where=(xx < -percentile) | (xx > percentile), alpha=0.5, color='#800080')
plt.title(f"Distribución de t con {nobs - 2} grados de libertad")
plt.show()

Concretamente, la probabilidad para el estadístico $t$ que has obtenido es muy muy baja.


In [ ]:
# p value for coefficient, extracted from
# t distribution
stats.t.sf(t_stat_x, nobs - 2) * 2